Importujemy najpierw potrzebne pakiety

In [26]:
import numpy as np
import scipy.io.wavfile
import scipy.signal

In [25]:
def wczytywanie_piosenki(nazwa_piosenki = "song.txt"):
    # 
    #dac asserty i doctringa ''''''
    #okreslic jakos lokalizacje skad ma brac te pliki ze z tego samego folderu gdzie jest program z katalogu 
    # odpowiedniego dla utworu - jesli w ogole, bo jedno to wczytanie pliku o takiej nazwie, a drugie skad - 
    # chyba automatycznie wczytuje z miejsca skad uruchomil sie python??
    """
    bedzie wczytywac zawartosc pliku song.txt do np.array zlozonego z macierzy n (liczba wierszy w song.txt) 
    na 1 - 1 wiersz bedzie odpowiadal jednemu trackowi - zapisane to bedzie jako string
    
    arg:
    nazwa_piosenki - nazwa pliku tekstowego wraz z rozszerzeniem, typ: str, assert - ze nie podano stringa
    reszta (ze podano zla sciezke zajmie sie genfromtxt)
    """
    
    import numpy as np

    
    song = np.genfromtxt(nazwa_piosenki,dtype='str')
    
    
    return song

In [27]:
a = wczytywanie_piosenki()
a
# czy nie bedzie trzeba wczytywac tego z folder /utwor1/track01.txt !!!!!!!!!!

array(['01', '02', '02', '01', '02'], 
      dtype='<U2')

In [28]:
def wczytywanie_sciezek(nazwa_piosenki):
    #dac asserty i doctringa ''''''
    #okreslic jakos lokalizacje skad ma brac te pliki ze z tego samego folderu gdzie jest program z katalogu 
    # odpowiedniego dla utworu
    
    # robimy z '01' napis 'track01.txt' by taki plik wczytac
    """
    bedziemy na podstawie pliku song.txt wczytywac tracki, typ: numpy.ndarray, i zapiszemy je jako jedna 
    macierz - zlaczymy w calosc
    
    
    
    assert, ze nie ten typ
    
    arg:
    nazwa_piosenki - np. array
    
    wyjscie:
    macierz typu np.ndarray kazdy wiersz to jedna cwiercnuta, elementy wiersza jako stringi
    """
    import numpy as np
    
    
    
    # ta funkcja zwraca np.ndarray, ktorego elementy sa juz posortowane; zakladam, ze sciezki ktore, sa podane
    # w song.txt maja numery odpowiadajace ich liczbie, tj. jak mam 3 rozne sciezki w song.txt to ich nazwy to
    # track01.txt, track02.txt, track03.txt, a nie np. track11.txt, track01.txt, track07.txt, to zalozenie
    # potrzebne dalej
    jakie_sciezki = np.unique(nazwa_piosenki)
    
    ile_sciezek = len(jakie_sciezki)
    
    #lista zwykla do trzymania sciezek, sciezka[0] - track01 (iterator trza cofnac)
    sciezki = []
    
    
    for ktora_sciezka in jakie_sciezki: # w iteratorze bierzemy napis "01" "02" itd. string!!!
        plik = ''.join(['track',ktora_sciezka,'.txt']) # tworzymy napis z nazwa pliku tracka,tj "track01.txt"
        #iterator ktora_sciezka przebiega po kolei numery (posortowane sa po unique)
        s = np.genfromtxt(plik, dtype='str')
        sciezki.append(s) # wczytujemy dane z niego
    
          
    piosenka = []
    for ktora_sciezka in nazwa_piosenki:       
        piosenka.append(sciezki[int(ktora_sciezka)-1])
    
    #zliczamy ile jest kanalow ( kolumn w trackach)
    ile_kanalow = sciezki[0].shape[1] # licze ile kolumn w plikach jest
    
    #zliczamy ile mamy cwiercnut
    n = 0
    for i in nazwa_piosenki:
        
        n += sciezki[int(i)-1].shape[0]
    
    #tworzymy z listy typ np.ndarray, ktory bedzie mial odpowiednie wymiary
    piosenka = np.array(piosenka).reshape(n, ile_kanalow)
    
    
    
    
    
    return piosenka
    
    
# machnąc w forze ze jak ktorys z trackow ma rozna liczbe kolumn to wywal blad albo
# sprawdzanie co jest w piluk to juz inna funkcja


In [29]:
import numpy as np
def wczytywanie_ustawien(plik_konfiguracyjny = "defs.txt"):
    """
    asserty 
    na razie zakladam, ze wszystkie parametry beda liczbami
    
    wczytywanie pliku z ustawieniami (pliku defs.txt)
    
    arg:
        nazwa pliku konfiguracyjnego, typ: string
        
    wyjscie:
        wartosci parametrow zapisane w slowniku, typ: dict
    
    """
    import re
    # wczytuje zawartowsc pliku (bez pierwszej i ostatniej linijki, jeden wiersz wyjsciowej macierzy, zawiera
    # nazwe parametru i jego wartosc, jako oddzielne elementy, zapisane jako stringi)
    ustawienia = np.genfromtxt(plik_konfiguracyjny, dtype = str,skip_header=1,skip_footer=1, delimiter=":")
    # tworze slownik, ktory bedzie przechowywal wartosci
    parametry = {}
    # tworze slownik
    # pozbywam się "" z key i przerabiam value na inty
    if len(ustawienia) == 2:
        parametry[re.sub('"','',ustawienia[0])] = int(ustawienia[1])
    else:
        for l in ustawienia:
        
            parametry[re.sub('"','',l[0])] = int(l[1])
    
    # zwracam gotowy slownik
    return parametry
wczytywanie_ustawien()

{'bpm': 120}

In [30]:
# NOWE PRZERÓBKA
def tworzenie_piosenki(macierz_piosenki, bpmy = 120):
    """
    asserty 
    glowna funkcja
    """
    
    t_cwiercnuty = 60 / bpmy
    n = macierz_piosenki.shape[0]
    kanaly = macierz_piosenki.shape[1]
    
    import numpy as np
    import scipy.io.wavfile
    import scipy.signal
    
    # utwor
    T = np.linspace(0, n*t_cwiercnuty, 44100*n*t_cwiercnuty)
        
    calosc = np.zeros((1,))
    for wiersz in range(0, 10): # ZMIEN TU NA N ZAMIAST 10
        
        sample = []
        dlugosci = []
        
        for i in range(0, kanaly):
            
            if macierz_piosenki[wiersz,i] == '--':
                sample.append(np.zeros((1,1)))    
            else:
                plik = ''.join(['sample', macierz_piosenki[wiersz,i],'.wav']) 
                frekw, t = scipy.io.wavfile.read(plik)
                sample.append(t)
                sample[i] = np.mean(sample[i], axis=1)/32767 # robi mono
                sample[i] = np.int16(sample[i]/max(np.abs(sample[i]))*32767)
                dlugosci.append(sample[i].shape[0])
        
        maksik = max(dlugosci)
        pusty = np.int16(np.zeros((len(sample), maksik)))
        for k in range(0, len(sample)):
            pusty[k][0:sample[k].shape[0]] = sample[k]
                
        cwiercnuta = sum(pusty)
        if (wiersz*t_cwiercnuty*44100 + maksik) > len(T):
            T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]= cwiercnuta[0:len(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)])]
        else:
            T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]= cwiercnuta
        

        
        
        
        #print(len(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]))
        #print(len(cwiercnuta))
        #print(type(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]))
        
    T= np.array(T, dtype=np.int16)
    return T


wierszyk = tworzenie_piosenki(wczytywanie_sciezek(a), b['bpm'])
wierszyk
# jak w to wcisnac bmpki?

/home/rexamine/anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


array([  68, -114,  121, ...,   39,   39,   40], dtype=int16)

In [31]:
import numpy as np
import scipy.io.wavfile
import scipy.signal
scipy.io.wavfile.write('test1.wav', 
                           44100,
                       wierszyk
                        )
!!aplay /home/rexamine/Desktop/test1.wav


["Playing WAVE '/home/rexamine/Desktop/test1.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

In [15]:
import os
os.getcwd()

'/home/rexamine/Desktop'